In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interact, SelectionSlider

In [2]:
# some statements to make the figures look nicer 
%config InlineBackend.figure_format='retina'
plt.rcParams.update({'axes.linewidth' : 1,
                     'ytick.major.width' : 1,
                     'ytick.minor.width' : 1,
                     'xtick.major.width' : 1,
                     'xtick.minor.width' : 1,
                     'xtick.labelsize': 10, 
                     'ytick.labelsize': 10,
                     'axes.labelsize': 12,
                     'font.family': 'Serif',
                     'figure.figsize': (10, 6)
                    })

In [3]:
N_bins = 50
r_edges = np.linspace(1, 25, N_bins+1)
r_avg = (3/4) * (r_edges[1:]**4 - r_edges[:-1]**4) / (r_edges[1:]**3 - r_edges[:-1]**3)
data = np.load('Corr_combos_data.npz')

In [4]:
def make_plot(sim, z, bhmass_min, fEdd_min, stmass_min, sf):
    mask = (data['sim'] == sim) & (data['z'] == z) & \
        (data['bhmass_min'] == bhmass_min) & (data['fEdd_min'] == fEdd_min) & \
        (data['stmass_min'] == stmass_min) & (data['sf'] == sf)

    if not np.any(mask):
        fig, (ax_xi, ax_R) = plt.subplots(2, 1, figsize=(6,8), sharex=True, constrained_layout=True)
        ax_xi.set_ylabel(r'$\xi$')
        ax_xi.set_title('2PCFs')
        ax_xi.set_xlim(r_edges[0],r_edges[-1])
        ax_xi.set_ylim(0,40)
        ax_R.set_xlabel(r'$r \, [{\rm cMpc} h^{-1}]$')
        ax_R.set_ylabel(r'$R$')
        ax_R.set_title('Bias ratio')
        ax_R.set_ylim(0,2)
        plt.show()

    else:
        xiqq_mean_arr = data['xiqq'][mask][0]
        xigg_mean_arr = data['xigg'][mask][0]
        xiqg_mean_arr = data['xiqg'][mask][0]
        R_mean_arr = data['R'][mask][0]
        xiqq_sigma_arr = data['xiqq_err'][mask][0]
        xigg_sigma_arr = data['xigg_err'][mask][0]
        xiqg_sigma_arr = data['xiqg_err'][mask][0]
        R_sigma_arr = data['R_err'][mask][0]
        constant = data['constant'][mask][0]
        constant_err = data['constant_err'][mask][0]

        fig, (ax_xi, ax_R) = plt.subplots(2, 1, figsize=(6,8), sharex=True, constrained_layout=True)
        
        ax_xi.errorbar(r_avg, xiqq_mean_arr, yerr=xiqq_sigma_arr, \
                       fmt='o-', markersize=3.5, lw=1, elinewidth=2.5, capsize=3, alpha=0.75, \
                       color='xkcd:royal blue', label=r'$\xi_{qq}$')
        ax_xi.errorbar(r_avg, xigg_mean_arr, yerr=xigg_sigma_arr, \
                       fmt='o-', markersize=3.5, lw=1, elinewidth=2.5, capsize=3, alpha=0.75, \
                       color='xkcd:turquoise', label=r'$\xi_{gg}$')
        ax_xi.errorbar(r_avg, xiqg_mean_arr, yerr=xiqg_sigma_arr, \
                       fmt='o-', markersize=3.5, lw=1, elinewidth=2.5, capsize=3, alpha=0.75, \
                       color='xkcd:magenta', label=r'$\xi_{qg}$')
        ax_xi.legend(loc='upper right')
        ax_xi.set_ylabel(r'$\xi$')
        ax_xi.set_title('2PCFs')
        ax_xi.set_xlim(r_edges[0],r_edges[-1])
        ax_xi.set_ylim(0,40)
        
        ax_R.errorbar(r_avg, R_mean_arr, yerr=R_sigma_arr, \
                      fmt='o-', markersize=3.5, lw=1, elinewidth=2.5, capsize=3, alpha=0.75, color='xkcd:royal blue')
        ax_R.axhline(constant, ls='dashed', lw=1.5, color='xkcd:magenta', \
                     label=f'Best fit: $R = {np.round(constant,3)} \\pm {np.round(constant_err,3)}$')
        ax_R.fill_between([r_edges[0], r_edges[-1]], constant-constant_err, constant+constant_err, \
                          alpha=0.45, color='xkcd:magenta')
        ax_R.legend(loc='lower center')
        ax_R.set_xlabel(r'$r \, \, [{\rm cMpc} \, h^{-1}]$')
        ax_R.set_ylabel(r'$R = b_q / b_g$')
        ax_R.set_title('Bias ratio')
        ax_R.set_ylim(0,2)
        
        plt.show()

In [5]:
interact(make_plot, \
    sim = SelectionSlider(options=['TNG', 'ASTRID'], value='TNG', description='Simulation', \
                          style={'description_width':'150px'}, layout=ipywidgets.Layout(width='500px')), \
    z = SelectionSlider(options=[1, 2, 3], value=1, description='$z$', \
                        style={'description_width':'150px'}, layout=ipywidgets.Layout(width='500px')), \
    bhmass_min = SelectionSlider(options=[('7',10**(7)), ('7.5',10**(7.5)), ('8',10**(8)), ('8.5',10**(8.5)), ('9',10**(9))], \
                                 value=10**(7), description=r'$\log_{10}{(M_{BH,min}/{\rm M}_\odot)}$', \
                                 style={'description_width':'150px'}, layout=ipywidgets.Layout(width='500px')), \
    fEdd_min = SelectionSlider(options=[('No min',-1), (0.001,0.001), (0.01,0.01), (0.05,0.05), (0.1,0.1)], value=-1, description=r'$f_{Edd,min}$', \
                               style={'description_width':'150px'}, layout=ipywidgets.Layout(width='500px')), \
    stmass_min = SelectionSlider(options=[('10',10**(10)), ('10.25',10**(10.25)), ('10.5',10**(10.5)), ('10.75',10**(10.75)), ('11',10**(11))], \
                                 value=10**(10), description=r'$\log_{10}{(M_{\star,min}/{\rm M}_\odot)}$', \
                                 style={'description_width':'150px'}, layout=ipywidgets.Layout(width='500px')), \
    sf = SelectionSlider(options=[('SF',True), ('Q',False)], value=True, description='Galaxy type', \
                         style={'description_width':'150px'}, layout=ipywidgets.Layout(width='500px')));

interactive(children=(SelectionSlider(description='Simulation', layout=Layout(width='500px'), options=('TNG', …